In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Read in the data
lats = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lons.npy')

p_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/precip-accu_11-16_12.npy')
p_full = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_full/precip-accu_11-16_12.npy')
p_large = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_large/precip-accu_11-16_12.npy')
p_small = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_small/precip-accu_11-16_12.npy')
p_box = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_box/precip-accu_11-16_12.npy')
p_hole = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_hole/precip-accu_11-16_12.npy')

dp_full = p_full - p_ctl
dp_large = p_large - p_ctl
dp_small = p_small - p_ctl
dp_box = p_box - p_ctl
dp_hole = p_hole - p_ctl


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*6.

### Plot subdomain
min_lat = 31
max_lat = 49
min_lon = -79
max_lon = -61

### Cartopy projection
proj = crs.PlateCarree()

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],
                               ['bottomleft','bottommiddle','bottomright'],
                               ['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(7.2,5.4), dpi=600, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.left_labels=True

im1 = axd['topleft'].contourf(lons, lats, dp_full, levels=zlvls,
            cmap = get_cmap('BrBG'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topleft'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('Full')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
##############

# Add the gridlines
gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])

im1 = axd['topmiddle'].contourf(lons, lats, dp_box, levels=zlvls,
            cmap = get_cmap('BrBG'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topmiddle'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topmiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('Box')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
##############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])

im1 = axd['topright'].contourf(lons, lats, dp_small, levels=zlvls,
            cmap = get_cmap('BrBG'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topleft'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('Small')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
##############

axd['bottomleft'].axis('off')


##############
### BOTTOM MIDDLE
##############

# Add the gridlines
gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

im1 = axd['bottommiddle'].contourf(lons, lats, dp_hole, levels=zlvls,
            cmap = get_cmap('BrBG'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottommiddle'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottommiddle'].set_title('Hole')
axd['bottommiddle'].text(0.0, 1.0, '(d)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
##############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

im1 = axd['bottomright'].contourf(lons, lats, dp_large, levels=zlvls,
            cmap = get_cmap('BrBG'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomright'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('Large')
axd['bottomright'].text(0.0, 1.0, '(e)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/4km_exp_precip.png',bbox_inches='tight')
plt.show()
